
# **<span style="color:red">Model Interpretability with SHAP and LIME:</span>** 
 Interpreting a fine-tuned token classification model using **SHAP** (SHapley Additive exPlanations) and **LIME** (Local Interpretable Model-agnostic Explanations). Below is a breakdown of the key components:

---
## **<span style="color:red">1. Imports</span>**  
- **`torch`**: PyTorch for deep learning operations.
- **`shap`**: SHAP library for model interpretability.
- **`numpy`**: For numerical computations.
- **`lime.lime_text`**: LIME library for text interpretability.
- **`transformers`**: Hugging Face library for loading pre-trained models and tokenizers.

---
## **<span style="color:red">2. Load Fine-Tuned Model**</span>**  

- **`model_path`**: Path to the fine-tuned model.
- **`AutoModelForTokenClassification`**: Loads the pre-trained token classification model.
- **`AutoTokenizer`**: Loads the tokenizer associated with the model.

```python
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
import torch
import shap
import numpy as np
from lime.lime_text import LimeTextExplainer
from transformers import AutoModelForTokenClassification, AutoTokenizer
from typing import List, Dict, Any

# Constants
MODEL_PATH = "C:/Users/ibsan/Desktop/TenX/week-5/model_output/results/mbert-fine-tuned"
SAMPLE_TEXT = "በአዲስ �በባ ውስጥ አዲስ ስልክ በ 5000 ብር ይገኛል።"

# Load model and tokenizer
def load_model_and_tokenizer(model_path: str) -> tuple:
    """
    Loads a pre-trained model and tokenizer from the specified path.
    """
    try:
        model = AutoModelForTokenClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        return model, tokenizer
    except Exception as e:
        raise RuntimeError(f"Failed to load model or tokenizer: {e}")

# SHAP prediction function
def predict_for_shap(texts: List[str]) -> np.ndarray:
    """
    Predicts probabilities for SHAP analysis.
    """
    try:
        tokenized_inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            logits = model(**tokenized_inputs).logits
        probabilities = torch.softmax(logits, dim=-1).numpy()
        return probabilities.astype(np.float32)
    except Exception as e:
        raise RuntimeError(f"Prediction failed for SHAP: {e}")

# LIME prediction function
def predict_for_lime(texts: List[str]) -> np.ndarray:
    """
    Predicts probabilities for LIME analysis.
    """
    try:
        inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
        probabilities = torch.softmax(logits, dim=-1).numpy()
        return probabilities.astype(np.float32)
    except Exception as e:
        raise RuntimeError(f"Prediction failed for LIME: {e}")

# SHAP analysis
def perform_shap_analysis(text: str, model: Any, tokenizer: Any) -> None:
    """
    Performs SHAP analysis on the input text.
    """
    try:
        masker = shap.maskers.Text(tokenizer)
        explainer = shap.Explainer(predict_for_shap, masker)
        shap_values = explainer([text])
        print("SHAP Analysis:")
        shap.plots.text(shap_values)
    except Exception as e:
        print(f"SHAP analysis failed: {e}")

# LIME analysis
def perform_lime_analysis(text: str, model: Any, tokenizer: Any) -> None:
    """
    Performs LIME analysis on the input text.
    """
    try:
        explainer = LimeTextExplainer(class_names=model.config.id2label.values())
        exp = explainer.explain_instance(text, predict_for_lime, num_features=10, top_labels=5)
        print("\nLIME Analysis:")
        exp.show_in_notebook()
    except Exception as e:
        print(f"LIME analysis failed: {e}")

# Main function
def main() -> None:
    """
    Main function to load the model, tokenizer, and perform interpretability analysis.
    """
    try:
        # Load model and tokenizer
        model, tokenizer = load_model_and_tokenizer(MODEL_PATH)

        # Perform SHAP and LIME analysis
        perform_shap_analysis(SAMPLE_TEXT, model, tokenizer)
        perform_lime_analysis(SAMPLE_TEXT, model, tokenizer)

        # Interpretability report
        print("\nModel Interpretability Report:")
        print("1. SHAP Analysis: Highlights the importance of each token in the model's predictions.")
        print("2. LIME Analysis: Provides local explanations for specific predictions.")
        print("3. Recommendations: Add more diverse training data, fine-tune on overlapping entities, or experiment with different architectures.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()